In [1]:
# Instalación de librerias

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from tqdm import tqdm
from scipy.integrate import simps
import numpy as np

### Abrir documento a procesar

In [6]:
name = "RDE_H_2024.xlsx"
RDE_cycle = pd.read_excel(name)
RDE_cycle.columns

Index(['intervalo', 't_s', 'v_km_h', 'RPM', 'Altitude_msnm'], dtype='object')

### Gráfico global cel ciclo

In [10]:
#Calcular velocidad en m/s
RDE_cycle['v_m_s'] = RDE_cycle['v_km_h']/3.6
# Calcula el cambio en el tiempo
RDE_cycle['delta_tiempo'] = RDE_cycle['t_s'].diff().fillna(0)
# Calcula la distancia recorrida en cada intervalo de tiempo
RDE_cycle['distancia'] = RDE_cycle['v_m_s'] * RDE_cycle['delta_tiempo']
# Calcula la distancia acumulada en km
RDE_cycle['distancia_acumulada'] = (RDE_cycle['distancia'].cumsum())/1000 # en km

## Agregar gróafico
fig = go.Figure()
# Agregar alguna traza de datos
fig.add_trace(go.Scatter(y=RDE_cycle["v_km_h"],name='Speed',
                         mode="lines",line=dict(color='black',width=1.2)))
# Agregar la segunda traza de datos
fig.add_trace(go.Scatter(x=RDE_cycle["t_s"], y=RDE_cycle["distancia_acumulada"], name='Distance',
                         yaxis='y2',line=dict(color='Gray',width=1.2)))

xmax = len(RDE_cycle["t_s"])

# Agregar una línea horizontal vel
fig.add_shape(type="line",x0=0,y0=60, x1=xmax,y1=60,line=dict(color="Red",width=0.6,
        dash="dashdot",),)
fig.add_shape(type="line",x0=0,y0=90, x1=xmax,y1=90,line=dict(color="Red",width=0.6,
        dash="dashdot",),)
fig.add_shape(type="line",x0=0,y0=145, x1=xmax,y1=145,line=dict(color="Red",width=0.6,
        dash="dashdot"),)
fig.add_annotation(x=3000, y=145, text="Speed thresholds", showarrow=True, arrowhead=1)

# Actualizar el layout para incluir el segundo eje Y
fig.update_layout(legend=dict(yanchor="top",y=1.1,xanchor="left",x=0.01),plot_bgcolor='white',paper_bgcolor='white', autosize=False,
    width=900,height=400, title_text="Accumulated speed and distance profile - RDE",
    xaxis=dict(ticklen=40,showgrid=False,zeroline=True,zerolinecolor='black',zerolinewidth=1,
        title="Time [s]",title_standoff=10),
    yaxis=dict(ticklen=40,showgrid=False,zeroline=True,zerolinecolor='black',zerolinewidth=1,
               title='Speed [km/h]',title_standoff=10),
    yaxis2=dict(ticklen=40,showgrid=False,zeroline=True,zerolinecolor='black',zerolinewidth=1,
                title='Distance [km]',title_standoff=10,overlaying='y',side='right',showline=True,
                linecolor='black',linewidth=1))

fig.show()


In [11]:
# separara datos de Intervalos del RDE (Urbano, Rural, Autopista)
df_Urbano = RDE_cycle.loc[RDE_cycle['intervalo'] == 'Urban']
df_Rural = RDE_cycle.loc[RDE_cycle['intervalo'] == 'Rural']
df_Autopista = RDE_cycle.loc[RDE_cycle['intervalo'] == 'Motorway']

## Porcentaje de cada intervalo respecto a la distancia total del trayecto (ciclo RDE)
Dist_RDE = RDE_cycle.distancia_acumulada.max() # Distacia del ciclo [km]
Dist_Urbano = (df_Urbano.distancia_acumulada.max()-df_Urbano.distancia_acumulada.min()) # Distancia urban [km]
Dist_Urbano_p = (Dist_Urbano*100)/Dist_RDE # Porcentaje Urban de la distancia ciclo
Dist_Rural = (df_Rural.distancia_acumulada.max()-df_Rural.distancia_acumulada.min()) # Distancia Rural [km]
Dist_Rural_p = (Dist_Rural*100)/Dist_RDE # Porcentaje Rural de la distancia ciclo
Dist_Autopista = (df_Autopista.distancia_acumulada.max()-df_Autopista.distancia_acumulada.min()) # Distancia Motorway [km]
Dist_Autopista_p = (Dist_Autopista*100)/Dist_RDE # Porcentaje Motorway de la distancia ciclo
speed_mean_urb = df_Urbano.v_km_h.mean() # Velocidad media en Urban


# Definicion de validacion de variables de distancia en km y %
def verificar_rango(valor, rango_inferior, rango_superior):
    if rango_inferior <= valor <= rango_superior:
        print("Cumple con criterio ---> OK")
    else:
        print("NO cumple con criterio --->NOK")

print ("*****Validación", name, " segun Reglamento (UN) 168*****")
print ("")  
print ("9.1 **Tipos de intervalos de velocidad**")
print ("9.1.1 **Otros requisitos**")
print ("Velocidad media del intervalo urbano {:.2f} km/h".format(speed_mean_urb))
verificar_rango(speed_mean_urb, 15, 40)

speed_hight = df_Autopista[df_Autopista["v_km_h"]>100]
time_SH = (len(speed_hight["intervalo"]))/60 # en minutos
print ("Tiempo de velocidad superior a 100 km/h en intervalo Autopista {:.2f} min".format(time_SH))
if time_SH >= 5:
    print ("Cumple con criterio ---> OK")
else:
    print("NO cumple con criterio --->NOK")
print ("")  

print ("9.2 **Porcentajes de distancia requerida de los intervalos de velocidad del trayecto**")
print ("Intervalo URBANO {:.2f}%".format(Dist_Urbano_p))
verificar_rango(Dist_Urbano_p, 29, 44)
print ("Intervalo RURAL {:.2f}%".format(Dist_Rural_p))
verificar_rango(Dist_Rural_p, 23, 43)
print ("Intervalo AUTOPISTA {:.2f}%".format(Dist_Autopista_p))
verificar_rango(Dist_Autopista_p, 23, 43)
print ("Distancia minima recorrida de cada intervalo, de 16 km")
print ("Distancia recorrida intervalo URBANO {:.2f} km".format(Dist_Urbano))
verificar_rango(Dist_Urbano, 16, 145)
print ("Distancia recorrida intervalo RURAL {:.2f} km".format(Dist_Rural))
verificar_rango(Dist_Rural, 16, 145)
print ("Distancia recorrida intervalo AUTOPISTA {:.2f} km".format(Dist_Autopista))
verificar_rango(Dist_Autopista, 16, 145)

print ("")
print ("9.3.3 **La velocidad máxima no superar los 145 km/h, tiempo en Stop entre 6 y 30 del intervalo urbano, y duración ciclo entre 90 y 120 min**")
print ("Velocidad máxima en Motorway {:.2f} km/h".format(df_Autopista.v_km_h.max()))
if df_Autopista.v_km_h.max() <= 145:
    print("Cumple con criterio ---> OK")
else:
    print("NO cumple con criterio --->NOK")

total_stop = df_Urbano[df_Urbano["v_km_h"]<=1]
ts = len (total_stop["t_s"])
total_urban = len (df_Urbano["t_s"])
stop = (ts*100)/total_urban    
print ("El tiempo en Stop {:.2f} %".format(stop))    
verificar_rango(stop, 6, 30)

duration = (RDE_cycle.t_s.max())/60
print ("Duración del trayecto (ciclo) {:.2f} min".format(duration))    
verificar_rango(duration, 90, 120)     
    
### OJO CALCULAR PARA LOS PRIMEROS 90 SEGUNDOS  
print ("")
print ("9.3.4 **La velocidad media (incluyendo paradas) durante el periodo de arranque en frío debe situarse entre 15 y 40 km/h, y no superar 60 km/h**")

df_90_segundos = df_Urbano[df_Urbano['t_s'] <= 90]
speed_mean_urban = df_90_segundos.v_km_h.mean()
print("Velocidad media de intervalo Urbano {:.2f} km/h".format(speed_mean_urban))
verificar_rango(speed_mean_urban, 15, 40)

print("Velocidad máxima durante arranque en frío {:.2f} km/h".format(df_90_segundos.v_km_h.max()))
if df_90_segundos.v_km_h.max() < 60:
    print("Cumple con criterio ---> OK")
else:
    print("NO cumple con criterio --->NOK")

*****Validación RDE_H_2024.xlsx  segun Reglamento (UN) 168*****

9.1 **Tipos de intervalos de velocidad**
9.1.1 **Otros requisitos**
Velocidad media del intervalo urbano 30.99 km/h
Cumple con criterio ---> OK
Tiempo de velocidad superior a 100 km/h en intervalo Autopista 15.60 min
Cumple con criterio ---> OK

9.2 **Porcentajes de distancia requerida de los intervalos de velocidad del trayecto**
Intervalo URBANO 31.00%
Cumple con criterio ---> OK
Intervalo RURAL 34.88%
Cumple con criterio ---> OK
Intervalo AUTOPISTA 34.13%
Cumple con criterio ---> OK
Distancia minima recorrida de cada intervalo, de 16 km
Distancia recorrida intervalo URBANO 30.77 km
Cumple con criterio ---> OK
Distancia recorrida intervalo RURAL 34.62 km
Cumple con criterio ---> OK
Distancia recorrida intervalo AUTOPISTA 33.87 km
Cumple con criterio ---> OK

9.3.3 **La velocidad máxima no superar los 145 km/h, tiempo en Stop entre 6 y 30 del intervalo urbano, y duración ciclo entre 90 y 120 min**
Velocidad máxima en Mot

FileNotFoundError: [Errno 2] No such file or directory: 'datos_ciclo_conduccion.xlsx'